# Grant Receipt Analysis
## Austrian Vendor Compliance Check

This notebook analyzes receipts from a government grant with two key requirements:
1. At least **50% of investments** must be from **Austrian vendors** (ATU UID numbers)
2. Individual receipts must be **≥€50** to be fundable

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better-looking plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)

## Load Budget Plan

The budget plan contains personnel costs which count towards Austrian vendor compliance (confirmed by AWS).

In [ ]:
import json

# Load budget plan
with open('budget_plan.json', 'r') as f:
    budget = json.load(f)

# Extract key values
personnel_funded = budget['personnel']['total_personnel_funded']
investment_budget = budget['investment']['total_investment_funded']
austrian_threshold = budget['compliance']['austrian_vendor_threshold']

print("="*70)
print("BUDGET PLAN LOADED")
print("="*70)
print(f"\nPersonnel Costs (90% funded): €{personnel_funded:,.2f}")
print(f"  - Counts as Austrian spending (AWS confirmed)")
print(f"\nInvestment Budget (90% funded): €{investment_budget:,.2f}")
print(f"\nAustrian Vendor Threshold: {austrian_threshold*100:.0f}%")
print("="*70)

In [ ]:
# Load the receipts data (item-level)
df = pd.read_csv('receipts_data.csv')

# Convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Display the data
print(f"Total number of items: {len(df)}")
print(f"Total number of unique receipts: {df['Receipt_Number'].nunique()}")
print(f"\nData overview (first 10 items):")
df.head(10)

## Receipt-Level Summary

In [ ]:
# Create receipt-level summary (one row per receipt)
receipts = df.groupby('Receipt_Number').agg({
    'Date': 'first',
    'Vendor': 'first',
    'Category': 'first',
    'UID_Number': 'first',
    'Is_ATU': 'first',
    'Receipt_Total': 'first',
    'Is_Fundable': 'first',
    'Item_Description': lambda x: f"{len(x)} items"
}).reset_index()

receipts.columns = ['Receipt_Number', 'Date', 'Vendor', 'Category', 'UID_Number', 'Is_ATU', 'Total', 'Is_Fundable', 'Items']

print("\nReceipt-Level Summary:")
receipts.sort_values('Date')

## Fundability Analysis

In [ ]:
# Analyze fundable vs non-fundable receipts
fundable_total = receipts[receipts['Is_Fundable'] == 'Yes']['Total'].sum()
non_fundable_total = receipts[receipts['Is_Fundable'] == 'No']['Total'].sum()
total_all_receipts = receipts['Total'].sum()

fundable_count = len(receipts[receipts['Is_Fundable'] == 'Yes'])
non_fundable_count = len(receipts[receipts['Is_Fundable'] == 'No'])

print("="*70)
print("FUNDABILITY ANALYSIS (€50 Threshold)")
print("="*70)
print(f"\nFundable Receipts (≥€50):")
print(f"  Count: {fundable_count}")
print(f"  Total: €{fundable_total:,.2f}")
print(f"\nNon-Fundable Receipts (<€50):")
print(f"  Count: {non_fundable_count}")
print(f"  Total: €{non_fundable_total:,.2f} (CANNOT be submitted)")
print(f"\n⚠️  Loss due to <€50 receipts: €{non_fundable_total:,.2f}")
print("="*70)

if non_fundable_count > 0:
    print("\nNon-Fundable Receipts Detail:")
    print(receipts[receipts['Is_Fundable'] == 'No'][['Date', 'Vendor', 'Category', 'Total', 'Items']])

## Austrian Vendor Compliance (with Personnel Costs)

In [ ]:
# Calculate compliance using FUNDABLE receipts only + Personnel costs
fundable_receipts = receipts[receipts['Is_Fundable'] == 'Yes']

# Receipt-based spending
austrian_vendor_spending = fundable_receipts[fundable_receipts['Is_ATU'] == 'Yes']['Total'].sum()
non_austrian_vendor_spending = fundable_receipts[fundable_receipts['Is_ATU'] == 'No']['Total'].sum()
total_receipt_spending = fundable_total

# Include personnel costs (from budget_plan.json)
total_austrian_contribution = austrian_vendor_spending + personnel_funded
total_project_spending = total_receipt_spending + personnel_funded

# Calculate percentages
austrian_percentage = (total_austrian_contribution / total_project_spending) * 100 if total_project_spending > 0 else 0

print("="*70)
print("AUSTRIAN VENDOR COMPLIANCE REPORT")
print("="*70)

print(f"\n📋 RECEIPT SPENDING (Fundable Only)")
print(f"   Austrian Vendors (ATU):     €{austrian_vendor_spending:,.2f}")
print(f"   Non-Austrian Vendors:       €{non_austrian_vendor_spending:,.2f}")
print(f"   Total Receipt Spending:     €{total_receipt_spending:,.2f}")

print(f"\n👥 PERSONNEL COSTS (Austrian Residents)")
print(f"   Personnel (90% funded):     €{personnel_funded:,.2f}")
print(f"   (Counts as Austrian - AWS confirmed)")

print(f"\n📊 COMPLIANCE CALCULATION")
print(f"   Total Austrian Contribution:")
print(f"     ATU Vendors:              €{austrian_vendor_spending:,.2f}")
print(f"     + Personnel:              €{personnel_funded:,.2f}")
print(f"     = Total Austrian:         €{total_austrian_contribution:,.2f}")
print(f"\n   Total Project Spending:")
print(f"     Receipts:                 €{total_receipt_spending:,.2f}")
print(f"     + Personnel:              €{personnel_funded:,.2f}")
print(f"     = Total Project:          €{total_project_spending:,.2f}")

print(f"\n   Austrian Percentage: €{total_austrian_contribution:,.2f} / €{total_project_spending:,.2f} = {austrian_percentage:.2f}%")

print("\n" + "="*70)
if austrian_percentage >= 50:
    print(f"✅ COMPLIANT: {austrian_percentage:.2f}% ≥ 50% requirement")
else:
    shortfall = 50 - austrian_percentage
    # Calculate how much more Austrian spending needed
    # We need: (austrian + x) / (total + x) >= 0.5
    # Solving: austrian + x >= 0.5 * total + 0.5 * x
    # 0.5 * x >= 0.5 * total - austrian
    # x >= total - 2 * austrian
    amount_needed = total_project_spending - (2 * total_austrian_contribution)
    if amount_needed < 0:
        amount_needed = 0
    print(f"❌ NON-COMPLIANT: {austrian_percentage:.2f}% < 50% requirement")
    print(f"   Shortfall: {shortfall:.2f}%")
    print(f"   Additional Austrian spending needed: €{amount_needed:,.2f}")
print("="*70)

print(f"\n💰 BUDGET STATUS")
print(f"   Investment Budget:          €{investment_budget:,.2f}")
print(f"   Spent on Receipts:          €{total_receipt_spending:,.2f}")
print(f"   Remaining:                  €{investment_budget - total_receipt_spending:,.2f}")

## Item-Level Breakdown

In [ ]:
# Show all items grouped by receipt
print("\nAll Items (grouped by receipt):")
for receipt_num in df['Receipt_Number'].unique():
    receipt_df = df[df['Receipt_Number'] == receipt_num]
    receipt_info = receipt_df.iloc[0]
    print(f"\n{'='*70}")
    print(f"Receipt: {receipt_num}")
    print(f"Date: {receipt_info['Date'].strftime('%Y-%m-%d')} | Vendor: {receipt_info['Vendor']}")
    print(f"Total: €{receipt_info['Receipt_Total']:.2f} | Fundable: {receipt_info['Is_Fundable']} | Austrian: {receipt_info['Is_ATU']}")
    print(f"{'='*70}")
    for idx, item in receipt_df.iterrows():
        print(f"  • {item['Item_Description']}: €{item['Item_Amount_Exact']:.2f}")

## Category Analysis

In [ ]:
# Investment by category (using item amounts)
print("\nInvestment by Category:")
category_summary = df.groupby('Category')['Item_Amount_Exact'].agg(['sum', 'count', 'mean'])
category_summary.columns = ['Total (EUR)', 'Number of Items', 'Average (EUR)']
category_summary

## Visualizations

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. ATU Compliance (Fundable only)
ax1 = fig.add_subplot(gs[0, 0])
atu_amounts = [non_austrian_spending, austrian_spending]
colors = ['#e74c3c', '#2ecc71']
labels = ['Non-Austrian', 'Austrian (ATU)']
wedges, texts, autotexts = ax1.pie(atu_amounts, labels=labels, autopct='%1.1f%%',
                                    startangle=90, colors=colors, textprops={'fontsize': 11})
ax1.set_title('Austrian vs Non-Austrian\n(Fundable Receipts)', fontsize=13, fontweight='bold')

status_color = '#2ecc71' if austrian_percentage >= 50 else '#e74c3c'
status_text = f'{'COMPLIANT' if austrian_percentage >= 50 else 'NON-COMPLIANT'}\n{austrian_percentage:.1f}% Austrian'
ax1.text(0, -1.3, status_text, ha='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor=status_color, alpha=0.3))

# 2. Fundable vs Non-Fundable
ax2 = fig.add_subplot(gs[0, 1])
fundable_amounts = [non_fundable_total, fundable_total]
colors2 = ['#e74c3c', '#2ecc71']
labels2 = [f'<€50\n(€{non_fundable_total:.2f})', f'≥€50\n(€{fundable_total:.2f})']
ax2.pie(fundable_amounts, labels=labels2, autopct='%1.1f%%',
        startangle=90, colors=colors2, textprops={'fontsize': 11})
ax2.set_title('Fundable vs Non-Fundable Receipts', fontsize=13, fontweight='bold')

# 3. Category breakdown
ax3 = fig.add_subplot(gs[1, :])
category_totals = df.groupby('Category')['Item_Amount_Exact'].sum().sort_values(ascending=True)
bars = ax3.barh(range(len(category_totals)), category_totals.values, color='#3498db')
ax3.set_yticks(range(len(category_totals)))
ax3.set_yticklabels(category_totals.index)
ax3.set_xlabel('Amount (EUR)', fontsize=11)
ax3.set_title('Total Spending by Category', fontsize=13, fontweight='bold')
ax3.grid(axis='x', alpha=0.3)
for i, v in enumerate(category_totals.values):
    ax3.text(v + 5, i, f'€{v:.2f}', va='center', fontsize=10)

# 4. Vendor breakdown
ax4 = fig.add_subplot(gs[2, :])
vendor_totals = receipts.groupby('Vendor')['Total'].sum().sort_values(ascending=False)
bars = ax4.bar(range(len(vendor_totals)), vendor_totals.values)

# Color Austrian vendors green
for i, vendor in enumerate(vendor_totals.index):
    is_austrian = receipts[receipts['Vendor'] == vendor]['Is_ATU'].iloc[0] == 'Yes'
    bars[i].set_color('#2ecc71' if is_austrian else '#3498db')

ax4.set_xticks(range(len(vendor_totals)))
ax4.set_xticklabels(vendor_totals.index, rotation=45, ha='right')
ax4.set_ylabel('Amount (EUR)', fontsize=11)
ax4.set_title('Spending by Vendor (Green = Austrian ATU)', fontsize=13, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)

for i, v in enumerate(vendor_totals.values):
    ax4.text(i, v + 5, f'€{v:.0f}', ha='center', va='bottom', fontsize=10)

plt.show()

## Export Summary Report

In [ ]:
# Create a comprehensive summary report (with personnel costs)
# Calculate compliance percentage with personnel
compliance_austrian = austrian_vendor_spending + personnel_funded
compliance_total = total_receipt_spending + personnel_funded
compliance_percentage = (compliance_austrian / compliance_total) * 100 if compliance_total > 0 else 0

summary_report = f"""GRANT RECEIPT SUMMARY REPORT
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}

{'='*70}
OVERVIEW
{'='*70}
Total Receipts: {len(receipts)}
Total Items: {len(df)}
Total Amount (All Receipts): €{total_all_receipts:,.2f}

{'='*70}
FUNDABILITY (€50 Threshold)
{'='*70}
Fundable Receipts (≥€50): {fundable_count} receipts = €{fundable_total:,.2f}
Non-Fundable (<€50): {non_fundable_count} receipts = €{non_fundable_total:,.2f}
Amount Lost to Non-Fundable: €{non_fundable_total:,.2f}

{'='*70}
AUSTRIAN VENDOR COMPLIANCE (Including Personnel Costs)
{'='*70}

RECEIPT SPENDING (Fundable Only):
  Austrian Vendors (ATU):     €{austrian_vendor_spending:,.2f}
  Non-Austrian Vendors:       €{non_austrian_vendor_spending:,.2f}
  Total Receipt Spending:     €{total_receipt_spending:,.2f}

PERSONNEL COSTS (Austrian Residents):
  Personnel (90% funded):     €{personnel_funded:,.2f}
  Note: Counts as Austrian spending (AWS confirmed)

COMPLIANCE CALCULATION:
  Total Austrian Contribution:
    ATU Vendors:              €{austrian_vendor_spending:,.2f}
    + Personnel:              €{personnel_funded:,.2f}
    = Total Austrian:         €{compliance_austrian:,.2f}

  Total Project Spending:
    Receipts:                 €{total_receipt_spending:,.2f}
    + Personnel:              €{personnel_funded:,.2f}
    = Total Project:          €{compliance_total:,.2f}

  Austrian Percentage: {compliance_percentage:.2f}%

Status: {'COMPLIANT' if compliance_percentage >= 50 else 'NON-COMPLIANT'}
Required: >= 50% from Austrian sources

{'='*70}
BUDGET STATUS
{'='*70}
Investment Budget (90% funded): €{investment_budget:,.2f}
Spent on Receipts:              €{total_receipt_spending:,.2f}
Remaining:                      €{investment_budget - total_receipt_spending:,.2f}

{'='*70}
CATEGORY BREAKDOWN
{'='*70}
{category_summary.to_string()}

{'='*70}
AUSTRIAN VENDORS (ATU)
{'='*70}
{receipts[receipts['Is_ATU'] == 'Yes'][['Date', 'Vendor', 'UID_Number', 'Total']].to_string()}
"""

print(summary_report)

# Save to file
with open('receipt_summary_report.txt', 'w', encoding='utf-8') as f:
    f.write(summary_report)
    
print("\n✓ Summary report saved to 'receipt_summary_report.txt'")

## Generate Markdown Report with Visualizations

In [ ]:
# Save the comprehensive visualization
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(4, 2, hspace=0.4, wspace=0.3)

# Calculate compliance with personnel for visualizations
compliance_austrian = austrian_vendor_spending + personnel_funded
compliance_total = total_receipt_spending + personnel_funded
compliance_percentage = (compliance_austrian / compliance_total) * 100 if compliance_total > 0 else 0

# 1. ATU Compliance WITH Personnel (main compliance chart)
ax1 = fig.add_subplot(gs[0, 0])
compliance_amounts = [non_austrian_vendor_spending, compliance_austrian]
colors = ['#e74c3c', '#2ecc71']
labels = ['Non-Austrian\nReceipts', 'Austrian\n(ATU + Personnel)']
wedges, texts, autotexts = ax1.pie(compliance_amounts, labels=labels, autopct='%1.1f%%',
                                    startangle=90, colors=colors, textprops={'fontsize': 10})
ax1.set_title('Austrian Compliance\n(Including Personnel)', fontsize=12, fontweight='bold')

status_color = '#2ecc71' if compliance_percentage >= 50 else '#e74c3c'
status_text = f"{'COMPLIANT' if compliance_percentage >= 50 else 'NON-COMPLIANT'}\n{compliance_percentage:.1f}%"
ax1.text(0, -1.4, status_text, ha='center', fontsize=11,
         bbox=dict(boxstyle='round', facecolor=status_color, alpha=0.3))

# 2. Fundable vs Non-Fundable
ax2 = fig.add_subplot(gs[0, 1])
fundable_amounts = [non_fundable_total, fundable_total]
colors2 = ['#e74c3c', '#2ecc71']
labels2 = [f'<€50\n(€{non_fundable_total:.0f})', f'>=€50\n(€{fundable_total:.0f})']
ax2.pie(fundable_amounts, labels=labels2, autopct='%1.1f%%',
        startangle=90, colors=colors2, textprops={'fontsize': 10})
ax2.set_title('Fundable vs Non-Fundable Receipts', fontsize=12, fontweight='bold')

# 3. Austrian Contribution Breakdown (stacked bar)
ax3 = fig.add_subplot(gs[1, :])
categories = ['Austrian\nContribution', 'Non-Austrian\nReceipts']
atu_vals = [austrian_vendor_spending, 0]
personnel_vals = [personnel_funded, 0]
non_atu_vals = [0, non_austrian_vendor_spending]

x = range(len(categories))
width = 0.5
bars1 = ax3.bar(x, atu_vals, width, label=f'ATU Vendors (€{austrian_vendor_spending:,.0f})', color='#2ecc71')
bars2 = ax3.bar(x, personnel_vals, width, bottom=atu_vals, label=f'Personnel (€{personnel_funded:,.0f})', color='#27ae60')
bars3 = ax3.bar(x, non_atu_vals, width, label=f'Non-Austrian (€{non_austrian_vendor_spending:,.0f})', color='#e74c3c')

ax3.set_ylabel('Amount (EUR)', fontsize=11)
ax3.set_title('Austrian vs Non-Austrian Spending Breakdown', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(categories)
ax3.legend(loc='upper right')
ax3.grid(axis='y', alpha=0.3)

# Add total labels on bars
ax3.text(0, compliance_austrian + 100, f'€{compliance_austrian:,.0f}', ha='center', fontweight='bold')
ax3.text(1, non_austrian_vendor_spending + 100, f'€{non_austrian_vendor_spending:,.0f}', ha='center', fontweight='bold')

# 4. Category breakdown
ax4 = fig.add_subplot(gs[2, :])
category_totals = df.groupby('Category')['Item_Amount_Exact'].sum().sort_values(ascending=True)
bars = ax4.barh(range(len(category_totals)), category_totals.values, color='#3498db')
ax4.set_yticks(range(len(category_totals)))
ax4.set_yticklabels(category_totals.index)
ax4.set_xlabel('Amount (EUR)', fontsize=11)
ax4.set_title('Total Spending by Category', fontsize=12, fontweight='bold')
ax4.grid(axis='x', alpha=0.3)
for i, v in enumerate(category_totals.values):
    ax4.text(v + 20, i, f'€{v:.0f}', va='center', fontsize=10)

# 5. Vendor breakdown
ax5 = fig.add_subplot(gs[3, :])
vendor_totals = receipts.groupby('Vendor')['Total'].sum().sort_values(ascending=False)
bars = ax5.bar(range(len(vendor_totals)), vendor_totals.values)

# Color Austrian vendors green
for i, vendor in enumerate(vendor_totals.index):
    is_austrian = receipts[receipts['Vendor'] == vendor]['Is_ATU'].iloc[0] == 'Yes'
    bars[i].set_color('#2ecc71' if is_austrian else '#3498db')

ax5.set_xticks(range(len(vendor_totals)))
ax5.set_xticklabels(vendor_totals.index, rotation=45, ha='right')
ax5.set_ylabel('Amount (EUR)', fontsize=11)
ax5.set_title('Spending by Vendor (Green = Austrian ATU)', fontsize=12, fontweight='bold')
ax5.grid(axis='y', alpha=0.3)

for i, v in enumerate(vendor_totals.values):
    ax5.text(i, v + 20, f'€{v:.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('receipt_analysis_charts.png', dpi=150, bbox_inches='tight')
print('Visualization saved to receipt_analysis_charts.png')
plt.show()

# Generate Markdown Report with Personnel Costs
md = f"""# Grant Receipt Analysis Report

**Generated:** {pd.Timestamp.now().strftime("%Y-%m-%d %H:%M")}

---

## Executive Summary

| Metric | Value |
|--------|-------|
| Total Receipts | {len(receipts)} |
| Total Items | {len(df)} |
| Total Amount (All Receipts) | €{total_all_receipts:,.2f} |
| Fundable Amount | €{fundable_total:,.2f} |
| Non-Fundable Loss | €{non_fundable_total:,.2f} |

---

## Austrian Vendor Compliance (Including Personnel)

### Status: {"COMPLIANT" if compliance_percentage >= 50 else "NON-COMPLIANT"} ({compliance_percentage:.2f}%)

The 50% Austrian vendor rule includes personnel costs for Austrian residents (confirmed by AWS).

### Receipt Spending (Fundable Only)

| Source | Amount |
|--------|--------|
| Austrian Vendors (ATU) | €{austrian_vendor_spending:,.2f} |
| Non-Austrian Vendors | €{non_austrian_vendor_spending:,.2f} |
| **Total Receipt Spending** | **€{total_receipt_spending:,.2f}** |

### Personnel Costs (Austrian Residents)

| Source | Amount |
|--------|--------|
| Personnel (90% funded) | €{personnel_funded:,.2f} |
| *Note: Counts as Austrian* | |

### Compliance Calculation

| Component | Amount |
|-----------|--------|
| ATU Vendors | €{austrian_vendor_spending:,.2f} |
| + Personnel | €{personnel_funded:,.2f} |
| **= Total Austrian** | **€{compliance_austrian:,.2f}** |
| | |
| Receipts | €{total_receipt_spending:,.2f} |
| + Personnel | €{personnel_funded:,.2f} |
| **= Total Project** | **€{compliance_total:,.2f}** |
| | |
| **Austrian Percentage** | **{compliance_percentage:.2f}%** |

**Requirement:** >= 50% from Austrian sources

---

## Budget Status

| Metric | Amount |
|--------|--------|
| Investment Budget (90% funded) | €{investment_budget:,.2f} |
| Spent on Receipts | €{total_receipt_spending:,.2f} |
| Remaining | €{investment_budget - total_receipt_spending:,.2f} |

---

## Visualizations

![Receipt Analysis Charts](receipt_analysis_charts.png)

---

## Fundable Receipts (>=€50)

{receipts[receipts["Is_Fundable"] == "Yes"][["Date", "Vendor", "Category", "Total"]].to_markdown(index=False)}

---

## Non-Fundable Receipts (<€50)

{receipts[receipts["Is_Fundable"] == "No"][["Date", "Vendor", "Category", "Total"]].to_markdown(index=False) if non_fundable_count > 0 else "None"}

---

## Austrian Vendors (ATU)

{receipts[receipts["Is_ATU"] == "Yes"][["Date", "Vendor", "UID_Number", "Total"]].to_markdown(index=False)}

---

## Category Breakdown

{category_summary.to_markdown()}

---

*Report generated from receipts_data.csv and budget_plan.json*
"""

with open("RECEIPT_ANALYSIS_REPORT.md", "w", encoding="utf-8") as f:
    f.write(md)

print("Markdown report saved to RECEIPT_ANALYSIS_REPORT.md")

In [ ]:
(12960+14850)/2